# Reworking Inference Models for Efficiency

© 2018 The Authors. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

--- 

In [1]:
import sys
sys.path.insert(0, '../../')
import numpy as np
import pandas as pd
import pystan 
import bokeh.io
import bokeh.plotting
import tqdm
import mut.viz
import mut.thermo
import mut.bayes
import bebi103.viz
constants = mut.thermo.load_constants()
bokeh.io.output_notebook()

Loading BokehJS ...

The purpose of this notebook is to spruce up the inferential models to be more efficient as well as develop new predictive checks and vizualization techniques. 

## DNA Binding Energy Inference

In [2]:
# Load the data. 
data = pd.read_csv('../../data/csv/compiled_data.csv')

# Restrict to physically meaningful values
data = data[(data['fold_change'] >= -0.2) & (data['fold_change'] <= 1.2)]

# Restrict to a single mutant and operator
DNA_data = data[(data['class']=='DNA') & (data['operator']=='O2')]

# Load the stan model. 
model = mut.bayes.StanModel('../stan/DNA_binding_energy_induction.stan')

Found precompiled model. Loading...
finished!


In [3]:
# Loop through each DNA mutant and perform the inference. 
mutant_dfs = []
summary_dfs = []
for g, d in tqdm.tqdm(DNA_data.groupby(['mutant', 'repressors'])):
    # Assemble the data dictionary.
    data_dict = {'J':1,
                 'N': len(d),
                 'idx': np.ones(len(d)).astype(int),
                 'R': d['repressors'], 
                 'Nns': constants['Nns'],
                 'ep_ai': constants['ep_AI'],
                 'Ka': constants['Ka'],
                 'Ki': constants['Ki'], 
                 'n_sites': constants['n_sites'],
                 'c': d['IPTGuM'],
                 'fc': d['fold_change']}
    
    # Sample!
    samples, samples_df = model.sample(data_dict=data_dict)
    
    # Get the parameter names and rename 
    parnames = samples.unconstrained_param_names()
    new_names = {'{}[{}]'.format(m.split('.')[0], m.split('.')[1]):'{}'.format(m.split('.')[0]) for m in parnames} 
    samples_df.rename(columns=new_names, inplace=True)
    
    # Add identifiers
    samples_df['repressors'] = g[1]
    samples_df['mutant'] = g[0]
    samples_df['operator'] = d['operator'].unique()[0]
    
    # Compute the summarized dataframe
    _df = samples_df[['ep_RA', 'sigma', 'lp__']].copy()
    summary_df = mut.stats.compute_statistics(_df, logprob_name='lp__')
    summary_df['repressors'] = g[1]
    summary_df['mutant'] = g[0]
    summary_df['operator'] = d['operator'].unique()[0]
  
    # Add to storage vector
    mutant_dfs.append(samples_df) 
    summary_dfs.append(summary_df)   
    
# Combine and save to disk    
mutant_df = pd.concat(mutant_dfs, sort=False)
summary_df = pd.concat(summary_dfs, sort=False)
mutant_df.to_csv('../../data/csv/DNA_binding_energy_samples.csv', index=False)
summary_df.to_csv('../../data/csv/DNA_binding_energy_summary.csv', index=False)
print('Finished!')

100%|██████████| 12/12 [00:08<00:00,  1.57it/s]


Finished!


In [4]:
# Plot the contours. 
samples = mutant_df[(mutant_df['mutant']=='Q21M') & 
                   (mutant_df['repressors']==60)]
p = bebi103.viz.corner(samples, vars=['ep_RA', 'sigma'])
bokeh.io.show(p)

## $K_A$ and $K_I$ Inference

In [4]:
# Restrict data to inducer mutants
IND_data = data[data['class']=='IND']

# Load the stan model. 
model = mut.bayes.StanModel('../stan/KaKi_fitting.stan', force_compile=True)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ebe7c4e051d4a4f843b70c03849f8159 NOW.


Precompiled model not found. Compiling model...


/anaconda3/lib/python3.6/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /var/folders/2q/lvh2zsws3lxckq8xtkn_84z80000gn/T/tmp_dwsdjtc/stanfit4anon_model_ebe7c4e051d4a4f843b70c03849f8159_7483483976972036041.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


KeyboardInterrupt: 

In [62]:
# Loop through each DNA mutant and perform the inference. 
mutant_dfs = []
summary_dfs = []
for g, d in tqdm.tqdm(IND_data.groupby(['mutant', 'operator'])):
    # Assemble the data dictionary.
    data_dict = {'J':1,
                 'N': len(d),
                 'idx': np.ones(len(d)).astype(int),
                 'R': d['repressors'], 
                 'Nns': constants['Nns'],
                 'ep_AI': constants['ep_AI'],
                 'ep_RA': constants[g[1]],
                 'n_sites': constants['n_sites'],
                 'c': d['IPTGuM'],
                 'fc': d['fold_change']}
    
    # Sample!
    samples, samples_df = model.sample(data_dict=data_dict, control=dict(adapt_delta=0.99))
    
    # Get the parameter names and rename 
    parnames = samples.unconstrained_param_names()
    new_names = {'{}[{}]'.format(m.split('.')[0], m.split('.')[1]):'{}'.format(m.split('.')[0]) for m in parnames} 
    samples_df.rename(columns=new_names, inplace=True)
    
    # Add identifiers
    samples_df['operator'] = g[1]
    samples_df['repressors'] = d['repressors'].unique()[0]
    samples_df['mutant'] = g[0]
    
    # Compute the summarized dataframe
    _df = samples_df[['Ka', 'Ki', 'sigma', 'lp__']].copy()
    summary_df = mut.stats.compute_statistics(_df, logprob_name='lp__')
    summary_df['repressors'] = d['repressors'].unique()[0]
    summary_df['mutant'] = g[0]
    summary_df['operator'] = g[1]
     
    # Add to storage vector
    mutant_dfs.append(samples_df)  
    summary_dfs.append(summary_df)   
    
# Combine and save to disk    
mutant_df = pd.concat(mutant_dfs, sort=False)
summary_df = pd.concat(summary_dfs, sort=False)
mutant_df.to_csv('../../data/csv/KaKi_only_samples.csv', index=False)
summary_df.to_csv('../../data/csv/KaKi_only_summary.csv', index=False)
print('Finished!')

100%|██████████| 10/10 [00:43<00:00,  3.66s/it]


Finished!


In [63]:
# Plot the contours. 
samples = mutant_df[(mutant_df['mutant']=='F164T') &
                    (mutant_df['operator']=='O2')]
                   
p = bebi103.viz.corner(samples, vars=['Ka', 'Ki', 'sigma'], datashade=True)
bokeh.io.show(p)

### $K_A$, $K_I$, and $\Delta\varepsilon_{AI}$ Inference

In [5]:
# Load the stan model
model = mut.bayes.StanModel('../stan/KaKi_epAI_fitting.stan')

# Loop through each DNA mutant and perform the inference. 
mutant_dfs = []
summary_dfs = []
for g, d in tqdm.tqdm(IND_data.groupby(['mutant', 'operator'])):
    # Assemble the data dictionary.
    data_dict = {'J':1,
                 'N': len(d),
                 'idx': np.ones(len(d)).astype(int),
                 'R': d['repressors'], 
                 'Nns': constants['Nns'],
                 'ep_RA': constants[g[1]],
                 'n_sites': constants['n_sites'],
                 'c': d['IPTGuM'],
                 'fc': d['fold_change']}
    # Sample!
    samples, samples_df = model.sample(data_dict=data_dict, iter=4000, 
                                       control=dict(adapt_delta=0.995, max_treedepth=11))
    # Get the parameter names and rename 
    parnames = samples.unconstrained_param_names()
    new_names = {'{}[{}]'.format(m.split('.')[0], m.split('.')[1]):'{}'.format(m.split('.')[0]) for m in parnames} 
    samples_df.rename(columns=new_names, inplace=True)
    
    # Add identifiers
    samples_df['operator'] = g[1]
    samples_df['repressors'] = d['repressors'].unique()[0]
    samples_df['mutant'] = g[0]
    _df = samples_df[['Ka', 'Ki', 'ep_AI', 'sigma', 'lp__']].copy()
    summary_df = mut.stats.compute_statistics(_df, logprob_name='lp__')
    summary_df['repressors'] = d['repressors'].unique()[0]
    summary_df['mutant'] = g[0]
    summary_df['operator'] = g[1]
    
    # Add to storage vector
    mutant_dfs.append(samples_df) 
    summary_dfs.append(summary_df)
    
# Combine and save to disk    
mutant_df = pd.concat(mutant_dfs, sort=False)
summary_df = pd.concat(summary_dfs, sort=False)
mutant_df.to_csv('../../data/csv/KaKi_epAI_samples.csv', index=False)
summary_df.to_csv('../../data/csv/KaKi_epAI_summary.csv', index=False)
print('Finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

Found precompiled model. Loading...
finished!


100%|██████████| 10/10 [01:40<00:00,  9.35s/it]


Finished!


## $K_A$, $K_I$, and $R$ inference

In [65]:
# Load the stan model
model = mut.bayes.StanModel('../stan/KaKi_R_fitting.stan')

# Loop through each DNA mutant and perform the inference. 
mutant_dfs = []
summary_dfs = []
for g, d in tqdm.tqdm(IND_data.groupby(['mutant', 'operator'])):
    # Assemble the data dictionary.
    data_dict = {'J':1,
                 'N': len(d),
                 'idx': np.ones(len(d)).astype(int),
                 'Nns': constants['Nns'],
                 'ep_RA': constants[g[1]],
                 'ep_AI': constants['ep_AI'],
                 'n_sites': constants['n_sites'],
                 'c': d['IPTGuM'],
                 'fc': d['fold_change']}
    # Sample!
    samples, samples_df = model.sample(data_dict=data_dict,
                                      control=dict(adapt_delta=0.99))

    # Get the parameter names and rename 
    parnames = samples.unconstrained_param_names()
    new_names = {'{}[{}]'.format(m.split('.')[0], m.split('.')[1]):'{}'.format(m.split('.')[0]) for m in parnames} 
    samples_df.rename(columns=new_names, inplace=True)
    
    # Add identifiers
    samples_df['operator'] = g[1]
    samples_df['repressors'] = d['repressors'].unique()[0]
    samples_df['mutant'] = g[0]
    _df = samples_df[['Ka', 'Ki', 'R', 'sigma', 'lp__']].copy()
    summary_df = mut.stats.compute_statistics(_df, logprob_name='lp__')
    summary_df['repressors'] = d['repressors'].unique()[0]
    summary_df['mutant'] = g[0]
    summary_df['operator'] = g[1]
    
    # Add to storage vector
    mutant_dfs.append(samples_df) 
    summary_dfs.append(summary_df)
    
# Combine and save to disk    
mutant_df = pd.concat(mutant_dfs, sort=False)
summary_df = pd.concat(summary_dfs, sort=False)
mutant_df.to_csv('../../data/csv/KaKi_R_samples.csv', index=False)
summary_df.to_csv('../../data/csv/KaKi_R_summary.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

Found precompiled model. Loading...
finished!


100%|██████████| 10/10 [00:37<00:00,  3.90s/it]


In [67]:
samples = mutant_df[(mutant_df['mutant']=='Q294V') &
                    (mutant_df['operator']=='O2')]
     
p = bebi103.viz.corner(samples, vars=['R', 'Ka', 'Ki', 'sigma'])
bokeh.io.show(p)